In [27]:
#imports
import time
import numpy as np
import pandas as pd
import pickle
import json
from pandas.io.json import json_normalize

In [54]:
#global constants
num_predictions = 50

In [40]:
load_path = 'saved_content/'

In [14]:
Art_to_ind_dict = {}
with open(load_path + 'art_to_ind_dict.pkl', 'rb') as f:
    Art_to_ind_dict = pickle.load(f)
    f.close()

In [15]:
track_uri_count_dict = {}
with open(load_path + 'track_uri_count_dict.pkl', 'rb') as f:
    track_uri_count_dict = pickle.load(f)
    f.close()

In [16]:
Artist_sim_weights = np.load(load_path + 'Artist_sim_weights.npy')

In [56]:
tracks_df = pd.read_csv(load_path + 'tracks_df.csv')
len(tracks_df)

89

In [21]:
def get_cagh_score(playlist, track):
    
    if track['artist_uri'] in Art_to_ind_dict:
        track_artist_idx = Art_to_ind_dict[track['artist_uri']]  # track artist index in artists_uri_train
    else:
        return 0
    
    if track['track_uri'] in track_uri_count_dict:
        count1 = track_uri_count_dict[track['track_uri']]
    else:
        return 0
    
    artists_test = np.array([t['artist_uri'] for t in playlist.tracks])   # name of all artists in playlist
    unique_artists_test = np.unique(artists_test)
    Ah = [Art_to_ind_dict[art] for art in unique_artists_test if art in Art_to_ind_dict] #Keep the indices of artists in the training dataset
    score = 0
    
    for artist_idx in Ah: 
        sim = Artist_sim_weights[artist_idx][track_artist_idx]
        score = score + (sim * count1)
    return score

In [22]:
def ndcg_score(prediction, truth):

    df_score = pd.DataFrame()
    df_score['predictionUri']=prediction
    df_score['trueUri']=truth
    
    ranking=[]
    ntracks = len(truth)
    #print ntracks
    
    for index,row in df_score.iterrows():
        if row.predictionUri in df_score.trueUri.values:
            ranking.append(ntracks - df_score[df_score['trueUri']==row.predictionUri].index[0])
        else:
            ranking.append(0)
    
    #Calculating the score
    df_score['rel']=ranking
    df_score['ind']=df_score.index+2
    df_score['logVal']=np.log2(df_score['ind'])
    NDCGscore = (df_score['rel'].divide(df_score['logVal'])).sum()
    return NDCGscore

def get_r_precision(ground_truth, predictions):
    count = 0
    for prediction in predictions:
        if prediction in ground_truth:
            count += 1
    return float(count*1.0/len(ground_truth))

In [28]:
def create_test_df():
    global prediction_df
    global hidden_df
    num_hidden_tracks_test = 50
    file_name = './data/challenge_set.json'
    with open(file_name) as json_file:
        json_data = json.load(json_file)
        for i in range(len(json_data['playlists'])):
            try:
                json_data['playlists'][i]['tracks'] = json_data['playlists'][i]['tracks'][:-num_hidden_tracks_test]
            except:
                print('WARN: No. of tracks less than the number to be hidden, escaped!') 
                pass
        prediction_df = pd.concat([prediction_df, json_normalize(json_data['playlists'])])

    # Repeating the above code to create df from other half of test.
    with open(file_name) as json_file:
        json_data = json.load(json_file)
        for i in range(len(json_data['playlists'])):
            try:
                json_data['playlists'][i]['tracks'] = json_data['playlists'][i]['tracks'][-num_hidden_tracks_test:]
            except:
                print('WARN: No. of tracks less than the number to be hidden, escaped!') 
                pass
        hidden_df = pd.concat([hidden_df, json_normalize(json_data['playlists'])])


prediction_df = pd.DataFrame()
hidden_df = pd.DataFrame()
create_test_df()

In [65]:
count = 0
total = 0
for index1, row1 in prediction_df.iterrows():
    predictions = []
    count += 1
    if count == 100:
        break
    for index2, row2 in tracks_df.iterrows():
        score = get_cagh_score(row1, row2)
        predictions.append([row2['track_name'], score])
    sorted_preds = sorted(predictions, key=lambda tup: (-tup[1]))   
    
    predictionLabels = [x[0] for x in sorted_preds]
    r_precision = get_r_precision(set([track['track_name'] for track in hidden_df.loc[index1]['tracks']]), predictionLabels[:num_predictions])
    print "r_precision is", r_precision
    
    if r_precision >= 0.1:
        total += 1
    
    #print "Predictions for playlist", row1['pid'], ":"
    #print sorted_preds[:num_predictions]    

    print "NDCG score is: ", ndcg_score(predictionLabels[:50], [track['track_name'] for track in hidden_df.loc[index1]['tracks']][:50])
    print "**********************************************************************************"
print total

r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.06
NDCG score is:  28.6289987528
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************


C:\Users\Sandeep\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if sys.path[0] == '':


r_precision is 0.02
NDCG score is:  26.49904965
**********************************************************************************
r_precision is 0.02
NDCG score is:  5.87741858728
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.02
NDCG score is:  4.33597239477
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
****************************************************************************

r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.0
NDCG score is:  0.0
**********************************************************************************
r_precision is 0.02
NDCG score is:  2.15338279037
**********************************************************************************
r_precision i